In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image


In [2]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text



In [3]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),

    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [4]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)




In [5]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 0.5
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [6]:

path_to_file = tf.keras.utils.get_file('littleprince_djvu.txt', 'https://archive.org/stream/TheLittlePrince-English/littleprince_djvu.txt')

text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)
embedding_dim = 300
rnn_units = 1024
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)
checkpoint_dir = './training'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
EPOCHS=50
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Epoch 1/50
23/23 [==============================] - 2s 105ms/step - loss: 3.4274
Epoch 2/50
23/23 [==============================] - 2s 66ms/step - loss: 2.8022
Epoch 3/50
23/23 [==============================] - 2s 66ms/step - loss: 2.4324
Epoch 4/50
23/23 [==============================] - 2s 67ms/step - loss: 2.1816
Epoch 5/50
23/23 [==============================] - 2s 67ms/step - loss: 1.9887
Epoch 6/50
23/23 [==============================] - 2s 67ms/step - loss: 1.8271
Epoch 7/50
23/23 [==============================] - 2s 65ms/step - loss: 1.6742
Epoch 8/50
23/23 [==============================] - 2s 65ms/step - loss: 1.5643
Epoch 9/50
23/23 [==============================] - 2s 67ms/step - loss: 1.4681
Epoch 10/50
23/23 [==============================] - 2s 68ms/step - loss: 1.3760
Epoch 11/50
23/23 [==============================] - 2s 67ms/step - loss: 1.3008
Epoch 12/50
23/23 [==============================] - 2s 66ms/step - loss: 1.2300
Epoch 13/50
23/23 [=================

In [7]:
print(generate_text(model, start_string=u"he say"))


W1020 20:20:33.178035 140297596929856 deprecation.py:323] From <ipython-input-5-5b162739346a>:12: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


he say to myself: âAt some moment or other one is absent-minded, and that is 
enough! On some one little stratagems. Flowers are so inconsistent! But I was too young to know how to take pleasure in all her grace. 

âI am unlucky,â said the lamplighter. âGood morning.â And he put out his lamp. 

Then he mopped his forehead with a handkerchief decorated with the desire to awaken. Then this little seed will stretch 
itself and begin, timidly at first, or a plant,â remarked the little prince, and added to himself, 
âThis flower is that of the boa constrictor from the outside and the boa constrictor from the outside and the boa constrictor from the inside, since I was a little boy who ims âAnd when your sorrow is comforted (time soothes all sorrows) you will be 
thires are to find them. The wind blows them away. Now how my ram. I felt hear him an according to 
think of my needsâ And the little prince, completely abashed. 

âHum! Hum!â replied the king. âThen I... I o